In [ ]:
import numpy as np
import math 

In [ ]:
def get_max_preds(heatmaps):
    """Get keypoint predictions from score maps.
    Note:
        batch_size: N
        num_keypoints: K
        heatmap height: H
        heatmap width: W
    Args:
        heatmaps (np.ndarray[N, K, H, W]): model predicted heatmaps.
    Returns:
        tuple: A tuple containing aggregated results.
        - preds (np.ndarray[N, K, 2]): Predicted keypoint location.
        - maxvals (np.ndarray[N, K, 1]): Scores (confidence) of the keypoints.
    """
    assert isinstance(heatmaps,
                      np.ndarray), ('heatmaps should be numpy.ndarray')
    assert heatmaps.ndim == 4, 'batch_images should be 4-ndim'

    N, K, _, W = heatmaps.shape
    heatmaps_reshaped = heatmaps.reshape((N, K, -1))
    idx = np.argmax(heatmaps_reshaped, 2).reshape((N, K, 1))
    maxvals = np.amax(heatmaps_reshaped, 2).reshape((N, K, 1))

    preds = np.tile(idx, (1, 1, 2)).astype(np.float32)
    preds[:, :, 0] = preds[:, :, 0] % W
    preds[:, :, 1] = preds[:, :, 1] // W

    preds = np.where(np.tile(maxvals, (1, 1, 2)) > 0.0, preds, -1)
    return preds, maxvals

def calc_dists(preds, target, normalize, use_zero=False):
    preds = preds.astype(np.float32)
    target = target.astype(np.float32)
    normalize = normalize.astype(np.float32)
    dists = np.zeros(preds.size(1), preds.size(0))
    if use_zero:
        boundary = 0
    else:
        boundary = 1
    for n in range(preds.shape[0]):
        for c in range(preds.shape[1]):
            if target[n,c,0] > boundary and target[n, c, 1] > boundary:
                dists[c, n] = np.linalg.norm((preds[n,c,:]- target[n,c,:])/normalize[n]) # axis ricavato da solo
            else:
                dists[c, n] = -1
    return dists

def accuracy(output, target, idkp, thr=0.5):
  ''' Calculate accuracy according to PCK, but uses ground truth heatmap rather than x,y locations
        First value to be returned is average accuracy across 'idxs', followed by individual accuracies
    '''
  preds = get_max_preds(output)
  gts = get_max_preds(target)
  norm = np.ones(preds.shape[0])*output.shape[3]/10

  dists = calc_dists(preds, gts, norm)

  acc = np.zeros(len(idxs)+1)
  avg_acc = 0
  cnt = 0

  for i in range(len(idxs)):
    acc[i+1] = dist_acc(dists[idxs[i]])
    if acc[i+1] >= 0: 
      avg_acc = avg_acc + acc[i+1]
      cnt += 1
            
  if cnt != 0:  
    acc[0] = avg_acc / cnt

  return acc